# On trigger signal acquisition

## Description

This example shows how to acquire 16k samples of signal on fast analog inputs.
Signal will be acquired when the internal trigger condition is meet.
Time length of the acquired signal depends on the time scale of a buffer
that can be set with a decimation factor.

TODO: describe some calculations

## Required hardware

- Red Pitaya
- Signal (function) generator

![wiring diagram](img/on_given_trigger_acquire_signal_on_fast_analog_input.png)

In [ ]:
from redpitaya import redpitaya
from time import sleep

rp = redpitaya()

# generator configuration
rp.GenReset()
rp.GenFreq(rp.CH_1, 100000.0)
rp.GenAmp(rp.CH_1, 1.0)
rp.GenWaveform(rp.CH_1, rp.WAVEFORM_SINE)
rp.GenOutEnable(rp.CH_1)

In [ ]:
# acquisition configuration
rp.AcqReset()
rp.AcqSetDecimationFactor(1)
rp.AcqSetTriggerLevel(rp.CH_1, 0.1)
rp.AcqSetTriggerDelay(0) # place trigger in the middle of the buffer

# start acquisition process
rp.AcqStart()
size = rp.AcqGetBufSize()
size = 2048
channels = (0, 1)

sleep(1)
# set trigger source to start acquisition
rp.AcqSetTriggerSrc(rp.TRIG_SRC_CHA_PE)

# wait in a loop for trigger state to chage from TRIG_STATE_WAITING
while rp.AcqGetTriggerSrc():
    pass
print('triggered')

# read data from FPGA FIFO into memory and display it
buff = [rp.AcqGetOldestDataV(ch, size) for ch in channels];

In [ ]:
import time

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
output_notebook()

In [ ]:
Fs = 125000000
N = size
x = np.arange(N)/Fs

colors = ('red', 'blue')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=500, plot_width=900, title="oscilloscope", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.yaxis.axis_label='voltage [V]'
#p.x_range=Range1d(0, N)
p.y_range=Range1d(-1.1, 1.1)
r = [p.line(x, buff[i], line_width=1, line_alpha=0.7, color=colors[i]) for i in channels]

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [ ]:
while True:
    rp.AcqStart()
    rp.AcqSetTriggerSrc(rp.TRIG_SRC_CHA_PE)
    while rp.AcqGetTriggerSrc(): pass
    buff = [rp.AcqGetOldestDataV(ch, size) for ch in channels];
#    buff = np.absolute(np.fft.fft(buff))
    for i in channels:
        r[i].data_source.data['y'] = buff[i]
    rp.AcqStart()
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
#    time.sleep(0.05)